# Load and select data: `db.load & db.select`

In [ ]:
import lamindb as ln

ln.nb.header()

## Load

Load data objects into memory via {class}`~lamindb.db.load`:

In [ ]:
dobject = ln.db.select.dobject(name="iris").first()

df = ln.db.load(dobject)

In [ ]:
df.head()

If no in-memory format can be found, returns the filepath:

In [ ]:
dobject = ln.db.select.dobject(name="paradisi05_laminopathic_nuclei").one()

ln.db.load(dobject)

To trace data back to its source, you can use the linked `dtransform` that generated the data. 

In [ ]:
dobject.dtransform_id

## Select

For simple frequent queries, LaminDB has a high-level select API {class}`~lamindb.db.select` based on [SQLModel](https://sqlmodel.tiangolo.com).

For complicated queries, you can use any SQL select. See below.

### Simple table queries

You can select any entity table in the database by its fields, for instance:

In [ ]:
ln.db.select.dobject(suffix=".feather").all()

The high-level select API by default returns a list of results:

In [ ]:
ln.db.select.species(taxon_id=10090).all()

You can call `.df()` to return a `DataFrame` instead.

In [ ]:
ln.db.select.species(taxon_id=10090).df()

If no constraints are passed, the select returns all rows in the table.

In [ ]:
ln.db.select.readout().all()

### Select data objects by linked entities

You can select data objects by fields that are not present in the `dobject` table via linked entities.

You can do this through providing a `where` dictionary.

In [ ]:
ln.db.select.dobject(suffix=".h5ad", where={"gene": {"symbol": "Actg1"}}).all()

`where` can filter conditions from multiple entities.

In [ ]:
from bioreadout import lookup

In [ ]:
ln.db.select.dobject(
    where=dict(
        gene=dict(ncbi_gene_id=66722),
        readout=dict(efo_id=lookup.readout.single_cell_RNA_sequencing),
    )
).df()

### SQL queries

You can construct arbitrary SQL queries via [SQLModel](https://sqlmodel.tiangolo.com).

For instance, let's select for a data source via the linked `dtransform`:

In [ ]:
with ln.db.session() as session:
    dtransform = session.get(ln.schema.core.dtransform, dobject.dtransform_id)

Inspecting the result we see that the dobject originates from a Jupyter Notebook.

In [ ]:
dtransform

In [ ]:
with ln.db.session() as session:
    jupynb = session.get(
        ln.schema.core.jupynb,
        (dtransform.jupynb_id, dtransform.jupynb_v),  # it's version "1" see jupynb_v
    )

In [ ]:
jupynb

Now we found the person who last edited the notebook!

In [ ]:
with ln.db.session() as session:
    user = session.get(ln.schema.core.user, jupynb.created_by)

In [ ]:
user